<a href="https://colab.research.google.com/github/fabiobento/dnn-course-2024-1/blob/main/00_course_folder/cert_prof_dl_intro/2%20-%20Introdu%C3%A7%C3%A3o%20%C3%A0%20vis%C3%A3o%20computacional/10%20-%20C1_W2_Lab_1_beyond_hello_world.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

adaptado de [Certificado Profissional Desenvolvedor do TensorFlow para DeepLearning.AI](https://www.coursera.org/professional-certificates/tensorflow-in-practice) de [Laurence Moroney](https://laurencemoroney.com/)

# Além do Hello World, um exemplo de visão computacional

No exercício anterior você viu um exemplo de um problema relativamente fácil: aprender a função `y=2x-1` a partir da relação entre `x` e `y`.

Mas o que dizer de um cenário em que escrever regras como essa é muito mais difícil, por exemplo, um problema de visão computacional?

Vamos dar uma olhada em um cenário em que você criará uma rede neural para reconhecer diferentes itens de vestuário, treinada a partir de um conjunto de dados com 10 tipos diferentes.

## Começando o código

Vamos começar com nossa importação do TensorFlow.

In [ ]:
import tensorflow as tf

print(tf.__version__)

O [Fashion MNIST dataset](https://github.com/zalandoresearch/fashion-mnist) é uma coleção de imagens de roupas em escala de cinza de 28x28 pixels. Cada imagem está associada a um rótulo, conforme mostrado nesta tabela⁉

| Rótulo | Descrição |
| --- | --- |
| 0 | T-shirt/top |
| 1 | Trouser |
| 2 | Pullover |
| 3 | Dress |
| 4 | Coat |
| 5 | Sandal |
| 6 | Shirt |
| 7 | Sneaker |
| 8 | Bag |
| 9 | Ankle boot |

Esse conjunto de dados está disponível diretamente na API [tf.keras.datasets](https://www.tensorflow.org/api_docs/python/tf/keras/datasets) e você o carrega da seguinte forma:

In [ ]:
# Carregar o conjunto de dados Fashion MNIST
fmnist = tf.keras.datasets.fashion_mnist

Ao chamar `load_data()` nesse objeto, você receberá duas tuplas com duas listas cada.

Esses serão os valores de treinamento e teste para as imagens que contêm os itens de vestuário e seus rótulos.


In [ ]:
# Carregar a divisão de treinamento e teste do conjunto de dados Fashion MNIST
(training_images, training_labels), (test_images, test_labels) = fmnist.load_data()

Qual é a aparência desses valores?
> Vamos imprimir uma imagem de treinamento (como uma imagem e uma matriz numérica) e um rótulo de treinamento para conferir.

Faça experiências com diferentes índices (variável `index`)na matriz.
> Por exemplo, dê uma olhada também no índice `42`.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Você pode colocar entre 0 e 59999 aqui
index = 0

# Definir o número de caracteres por linha ao imprimir
np.set_printoptions(linewidth=320)

# Imprimir o rótulo e a imagem
print(f'RÓTULO: {training_labels[index]}')
print(f'\nmatriz de pixels da imagem:\n {training_images[index]}')

# Visualize a imagem
plt.imshow(training_images[index])

Você perceberá que todos os valores do número estão entre 0 e 255.

Se estiver treinando uma rede neural, especialmente no processamento de imagens, por vários motivos, ela geralmente aprenderá melhor se você dimensionar todos os valores entre 0 e 1.
> É um processo chamado _normalização_ e, felizmente, em Python, é fácil normalizar uma matriz sem fazer loop.

Você faz isso da seguinte forma:

In [ ]:
# Normalize os valores de pixel das imagens de treinamento e teste
training_images  = training_images / 255.0
test_images = test_images / 255.0

Agora você deve estar se perguntando por que o conjunto de dados é dividido em dois:
* treinamento
* teste?

Lembra-se de que falamos anteriormente?
> A ideia é ter um conjunto de dados para treinamento e outro conjunto de dados que o modelo ainda não tenha visto.

Isso é usado para avaliar a qualidade do modelo na classificação de valores.

Vamos agora projetar o modelo

In [ ]:
# Criar o modelo de classificação
model = tf.keras.models.Sequential([tf.keras.layers.Flatten(), 
                                    tf.keras.layers.Dense(128, activation=tf.nn.relu), 
                                    tf.keras.layers.Dense(10, activation=tf.nn.softmax)])

[Sequential](https://keras.io/api/models/sequential/): Define uma sequência de camadas na rede neural.

[Flatten](https://keras.io/api/layers/reshaping_layers/flatten/): Lembra-se de quando nossas imagens eram uma matriz de 28x28 pixels quando você as imprimia? O Flatten simplesmente pega esse quadrado e o transforma em uma matriz unidimensional.

[Dense](https://keras.io/api/layers/core_layers/dense/): Adiciona uma camada de neurônios

Cada camada de neurônios precisa de uma [função de ativação](https://keras.io/api/layers/activations/) para dizer a eles o que fazer. Há muitas opções, mas, por enquanto, use apenas estas: 

[ReLU](https://keras.io/api/layers/activations/#relu-function) significa efetivamente:

```
se x > 0: 
  retorne x
senão: 
  retorne 0
```

Em outras palavras, ele só passa valores maiores que 0 para a próxima camada da rede.

[Softmax](https://keras.io/api/layers/activations/#softmax-function) recebe uma lista de valores e os dimensiona de modo que a soma de todos os elementos seja igual a 1.
> Quando aplicado aos resultados do modelo, você pode pensar nos valores dimensionados como a probabilidade dessa classe.
Por exemplo, em seu modelo de classificação que tem 10 unidades na camada densa de saída, ter o valor mais alto no `índice = 4` significa que o modelo está mais confiante de que a imagem de entrada da roupa é um casaco. Se estiver no índice = 5, então é uma sandália, e assim por diante. 

Veja o pequeno bloco de código abaixo que demonstra esses conceitos. Você também pode assistir a esta [aula de Andrew Ng](https://www.youtube.com/watch?v=LLux1SW--oM&ab_channel=DeepLearningAI) se quiser saber mais sobre a função Softmax e como os valores são computados.


In [ ]:
# Declare as entradas de amostra e converta-as em um tensor
inputs = np.array([[1.0, 3.0, 4.0, 2.0]])
inputs = tf.convert_to_tensor(inputs)
print(f'entrada para a função softmax: {inputs.numpy()}')

# Alimente as entradas para uma função de ativação softmax
outputs = tf.keras.activations.softmax(inputs)
print(f'saída da função softmax: {outputs.numpy()}')

# Obter a soma de todos os valores após o softmax
sum = tf.reduce_sum(outputs)
print(f'soma das saídas: {sum}')

# Obter a soma de todos os valores após o softmax
prediction = np.argmax(outputs)
print(f'classe com maior probabilidade: {prediction}')

A próxima coisa a fazer, agora que o modelo está definido, é construí-lo de fato.

Para isso, compile-o com um otimizador e uma função de perda como antes e, em seguida, treine-o chamando `model.fit()` para que ele ajuste os dados de treinamento aos rótulos de treinamento.

Ele descobrirá a relação entre os dados de treinamento e seus rótulos reais, de modo que, no futuro, se você tiver entradas que se pareçam com os dados de treinamento, ele poderá prever qual é o rótulo dessa entrada.

In [ ]:
model.compile(optimizer = tf.optimizers.Adam(),
              loss = 'sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(training_images, training_labels, epochs=5)

Quando o treinamento estiver concluído, você deverá ver um valor de acurácia no final da última época. Ele pode se parecer com algo como `0,9098`.

Isso indica que sua rede neural tem cerca de 91% de acurácia na classificação dos dados de treinamento.
> Ou seja, ela descobriu uma correspondência de padrão entre a imagem e os rótulos que funcionou 91% das vezes.

Não é ótimo, mas não é ruim, considerando que foi treinado por apenas 5 épocas e foi feito muito rapidamente.

Mas como ele funcionaria com dados não vistos?
> É por isso que temos as imagens de teste e os rótulos.

Podemos chamar [`model.evaluate()`](https://keras.io/api/models/model_training_apis/#evaluate-method) com esse conjunto de dados de teste como entradas e ele informará a perda e a precisão do modelo:

In [ ]:
# Avalie o modelo em dados não vistos
model.evaluate(test_images, test_labels)

Você pode esperar que a precisão aqui seja de cerca de `0,88`, o que significa que ela foi 88% precisa em todo o conjunto de teste.

Como era de se esperar, ele provavelmente não se sairia tão bem com dados *não vistos* quanto se saiu com dados nos quais foi treinado!

Ao longo desta matéria, você verá maneiras de melhorar isso. 

# Exercícios

Para explorar mais e aprofundar sua compreensão, experimente os exercícios abaixo:

### Exercício 1:
Para este primeiro exercício, execute o código abaixo:
* Ele cria um conjunto de classificações para cada uma das imagens de teste e, em seguida, imprime somente a primeira das classificações.
* A saída, depois de executá-lo, é uma lista de números. Por que você acha que isso acontece e o que esses números representam? 

In [ ]:
classifications = model.predict(test_images)

print(classifications[0])

**Dica:** tente executar `print(test_labels[0])` -- e você obterá um `9`. Isso ajuda você a entender por que essa lista tem a aparência que tem? 

In [ ]:
print(test_labels[0])

### QUIZ E1Q1: O que essa lista exibida por `print(classifications[0])` representa?


1.   São 10 valores aleatórios sem sentido
2.   São as 10 primeiras classificações que o computador fez
3.   É a probabilidade de esse item pertencer a cada uma das 10 classes


<details><summary>Clique para respostas</summary>
<p>

#### Resposta: 
A resposta correta é (3)

A saída do modelo é uma lista de 10 números. Esses números são a probabilidade de que o valor que está sendo classificado seja o valor correspondente (https://github.com/zalandoresearch/fashion-mnist#labels), ou seja, o primeiro valor da lista é a probabilidade de que a imagem seja "0" (T-shirt/top), o próximo é "1" (Trouser) etc. Observe que todas essas probabilidades são MUITO BAIXAS.

Para o índice 9 (Ankle boot), a probabilidade estava na casa dos 90, ou seja, a rede neural está nos dizendo que a imagem provavelmente é uma ankle boot.
</p>
</details>

### QUIZ E1Q2: Como você sabe que essa lista lhe diz que o item é uma ankle boot?


1.   Não há informações suficientes para responder a essa pergunta
2.   O 10º elemento da lista é o maior, e a ankle boot está identificada como 9
2.   A bota tem o rótulo 9, e há 0->9 elementos na lista.


<details><summary>Clique para resposta</summary>
<p>

#### Resposta
A resposta correta é (2). Tanto a lista quanto os rótulos são baseados em 0, portanto, o fato de a ankle boot ter o rótulo 9 significa que ela é a 10ª das 10 classes. O fato de a lista ter o 10º elemento com o valor mais alto significa que a rede neural previu que o item que está classificando é provavelmente uma ankle boot

</p>
</details>

### Exercício 2: 
Vamos agora examinar as camadas do seu modelo.

Faça experiências com valores diferentes para a camada densa com 512 neurônios.

Que resultados diferentes você obtém para perda, tempo de treinamento etc.? Por que você acha que esse é o caso? 

In [ ]:
fmnist = tf.keras.datasets.fashion_mnist

(training_images, training_labels) ,  (test_images, test_labels) = fmnist.load_data()

training_images = training_images/255.0
test_images = test_images/255.0

model = tf.keras.models.Sequential([tf.keras.layers.Flatten(),
                                    tf.keras.layers.Dense(512, activation=tf.nn.relu), # Tente fazer experimentos com essa camada
                                    tf.keras.layers.Dense(10, activation=tf.nn.softmax)])

model.compile(optimizer = 'adam',
              loss = 'sparse_categorical_crossentropy')

model.fit(training_images, training_labels, epochs=5)

model.evaluate(test_images, test_labels)

classifications = model.predict(test_images)

print(classifications[0])
print(test_labels[0])

### QUIZ E2Q1: Aumento para 1024 neurônios - Qual é o impacto?

1. O treinamento demora mais, mas é mais preciso
2. O treinamento é mais demorado, mas não há impacto na precisão
3. O treinamento leva o mesmo tempo, mas é mais preciso


<details><summary>Clique para resposta</summary>
<p>

#### Resposta
A resposta correta é (1): ao adicionar mais neurônios, temos de fazer mais cálculos, o que torna o processo mais lento, mas, nesse caso, eles têm um bom impacto: ficamos mais precisos. Isso não significa que seja sempre um caso de "mais é melhor", pois você pode atingir a lei dos retornos decrescentes ([_law of diminishing returns_](https://en.wikipedia.org/wiki/Diminishing_returns)) muito rapidamente!

</p>
</details>

### Exercício 3: 

### QUIZ E3Q1: O que aconteceria se você removesse a camada Flatten(). Por que você acha isso? 

<details><summary>Clique para Resposta</summary>
<p>

#### Answer
#### Resposta
Você recebe um erro sobre a forma dos dados. Isso pode parecer vago no momento, mas reforça a regra geral de que a primeira camada da rede deve ter o mesmo formato dos dados. No momento, nossos dados são imagens de 28x28 e 28 camadas de 28 neurônios seriam inviáveis, portanto, faz mais sentido "achatar" esses 28,28 em 784x1. Em vez de escrever todo o código para lidar com isso nós mesmos, adicionamos a camada Flatten() no início e, quando as matrizes forem carregadas no modelo posteriormente, elas serão automaticamente achatadas para nós.

</p>
</details>

In [ ]:
fmnist = tf.keras.datasets.fashion_mnist

(training_images, training_labels) ,  (test_images, test_labels) = fmnist.load_data()

training_images = training_images/255.0
test_images = test_images/255.0

model = tf.keras.models.Sequential([tf.keras.layers.Flatten(), #Try removing this layer
                                    tf.keras.layers.Dense(64, activation=tf.nn.relu),
                                    tf.keras.layers.Dense(10, activation=tf.nn.softmax)])

model.compile(optimizer = 'adam',
              loss = 'sparse_categorical_crossentropy')

model.fit(training_images, training_labels, epochs=5)

model.evaluate(test_images, test_labels)

classifications = model.predict(test_images)

print(classifications[0])
print(test_labels[0])

### Exercício 4: 

Considere as camadas finais (de saída). Por que há 10 delas? O que aconteceria se você tivesse uma quantidade diferente de 10? Por exemplo, tente treinar a rede com 5.

<details><summary>Clique para respostas</summary>
<p>

#### Resposta
Lembre-se do que você viu sobre a [Função Softmax](https://colab.research.google.com/github/fabiobento/dnn-course-2024-1/blob/main/00_course_folder/nn_adv/class_02/Laborat%C3%B3rios/C2_W2_SoftMax.ipynb)

Você recebe um erro assim que encontra um valor inesperado. Outra regra geral: o número de neurônios na última camada deve corresponder ao número de classes que você está classificando. Nesse caso, são os dígitos de 0 a 9, portanto, há 10 deles e, portanto, você deve ter 10 neurônios na camada final.

</p>
</details>

In [ ]:
fmnist = tf.keras.datasets.fashion_mnist

(training_images, training_labels) ,  (test_images, test_labels) = fmnist.load_data()

training_images = training_images/255.0
test_images = test_images/255.0

model = tf.keras.models.Sequential([tf.keras.layers.Flatten(),
                                    tf.keras.layers.Dense(64, activation=tf.nn.relu),
                                    tf.keras.layers.Dense(10, activation=tf.nn.softmax) # Tente fazer experimentos com essa camada
                                  ])

model.compile(optimizer = 'adam',
              loss = 'sparse_categorical_crossentropy')

model.fit(training_images, training_labels, epochs=5)

model.evaluate(test_images, test_labels)

classifications = model.predict(test_images)

print(classifications[0])
print(test_labels[0])

### Exercício 5: 

Considere os efeitos de camadas adicionais na rede. O que acontecerá se você adicionar outra camada entre a que tem 512 e a camada final com 10. 

<details><summary>Clique para Resposta</summary>
<p>

#### Resposta 
Não há um impacto significativo, porque esses dados são relativamente simples. Para dados muito mais complexos (incluindo imagens coloridas a serem classificadas como flores, que você verá mais a frente), geralmente são necessárias camadas extras. 


</p>
</details>

In [ ]:
fmnist = tf.keras.datasets.fashion_mnist

(training_images, training_labels) ,  (test_images, test_labels) = fmnist.load_data()

training_images = training_images/255.0
test_images = test_images/255.0

model = tf.keras.models.Sequential([tf.keras.layers.Flatten(),
                                    # Adicione uma camada aqui,
                                    tf.keras.layers.Dense(256, activation=tf.nn.relu),
                                    # Adicione uma camada aqui
                                  ])

model.compile(optimizer = 'adam',
              loss = 'sparse_categorical_crossentropy')

model.fit(training_images, training_labels, epochs=5)

model.evaluate(test_images, test_labels)

classifications = model.predict(test_images)

print(classifications[0])
print(test_labels[0])

### Exercício 6: 

### QUIZ E6Q1: Considere o impacto do treinamento por mais ou menos épocas.

- Experimente 15 épocas - você provavelmente obterá um modelo com uma perda muito melhor do que aquele com 5
- Experimente 30 épocas - você verá que o valor da perda diminui mais lentamente e, às vezes, aumenta. Provavelmente, você também verá que os resultados de `model.evaluate()` não melhoraram muito. Pode até ser um pouco pior.

Esse é um efeito colateral do "_overfitting_".
> Não faz sentido perder tempo treinando se não estiver melhorando a perda, certo? :)

In [ ]:
fmnist = tf.keras.datasets.fashion_mnist

(training_images, training_labels) ,  (test_images, test_labels) = fmnist.load_data()

training_images = training_images/255.0
test_images = test_images/255.0

model = tf.keras.models.Sequential([tf.keras.layers.Flatten(),
                                    tf.keras.layers.Dense(128, activation=tf.nn.relu),
                                    tf.keras.layers.Dense(10, activation=tf.nn.softmax)])

model.compile(optimizer = 'adam',
              loss = 'sparse_categorical_crossentropy')

model.fit(training_images, training_labels, epochs=15) # Experimente o número de épocas

model.evaluate(test_images, test_labels)


### Exercício 7: 

Antes de treinar, você normalizou os dados, passando de valores de 0 a 255 para valores de 0 a 1.
> Qual seria o impacto de remover isso?

Aqui está o código completo para tentar fazer isso.

***Por que você acha que obteve resultados diferentes?*** 
> Recorde o que conversamos sobre esse assunto no seguinte caderno: [Feature scaling e Taxa de Aprendizagem (Multi-Variável)](https://colab.research.google.com/github/fabiobento/dnn-course-2024-1/blob/main/00_course_folder/ml_intro/class_02/Laborat%C3%B3rios/C1_W2_Lab03_Feature_Scaling_and_Learning_Rate_Soln.ipynb)

In [ ]:
fmnist = tf.keras.datasets.fashion_mnist

(training_images, training_labels) ,  (test_images, test_labels) = fmnist.load_data()

training_images=training_images/255.0 # Experimente remover essa linha
test_images=test_images/255.0 # Experimente remover essa linha
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(512, activation=tf.nn.relu),
  tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')
model.fit(training_images, training_labels, epochs=5)
model.evaluate(test_images, test_labels)
classifications = model.predict(test_images)
print(classifications[0])
print(test_labels[0])

### Exercício 8: 

Anteriormente, quando você treinou por épocas extras, teve um problema em que sua perda ficava pior.
> Você pode ter pensado "não seria bom se eu pudesse interromper o treinamento quando atingisse um valor desejado?"

Ou seja, 60% de precisão pode ser suficiente para você e, se você atingir esse valor após 3 épocas, por que ficar esperando que ele termine muitas outras épocas se você tem os [_callbacks_](https://en.wikipedia.org/wiki/Callback_(computer_programming))!  :-)

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy') >= 0.6): # Experimente alterar esse valor
      print("\nAtingiu 60% de precisão, portanto, cancelou o treinamento!")
      self.model.stop_training = True

callbacks = myCallback()

fmnist = tf.keras.datasets.fashion_mnist
(training_images, training_labels) ,  (test_images, test_labels) = fmnist.load_data()

training_images=training_images/255.0
test_images=test_images/255.0
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(512, activation=tf.nn.relu),
  tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(training_images, training_labels, epochs=5, callbacks=[callbacks])
